MODS Phenotypes: Step 5. Emory Sub-Graph Mining
===

# `imports`

In [1]:
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import multiprocessing
import dill as pickle
from tqdm.auto import tqdm
from multiprocessing import Pool

n_cpus = 180

***TODO***: Instead of inserting the sys path to the local code, pip install the repo to a `mods` environment

In [2]:
sys.path.insert(0, "/opt/scratchspace/KLAB_SAIL/MODSPhenotypes/gSpan/")
import shlex

In [3]:
from src.gspan_mining.main import parse_args, main
from src.gspan_mining.data_processing import process_graph_data

In [4]:
site_name = 'emory'
run_id = '2023_07_29'

# 1. Process the saved `pd.DataFrame()` on disk to our `.gSpan` format.

In [30]:
data_fp = Path(f"/opt/scratchspace/KLAB_SAIL/MODSPhenotypes/data/{run_id}/{site_name}_sofa_scores_df.parquet")

In [14]:
output_dir = Path.home() / 'tmp' / 'gspan_output'
output_dir.mkdir(exist_ok=True, parents=True)

In [15]:
%%time
# TODO: benchmark the wall clock time on this call
file_paths = process_graph_data(data_fp, output_dir, n_cpus)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/55799 [00:00<?, ?it/s]

  0%|          | 0/55799 [00:00<?, ?it/s]

  0%|          | 0/55799 [00:00<?, ?it/s]

  0%|          | 0/55799 [00:00<?, ?it/s]

  0%|          | 0/55799 [00:00<?, ?it/s]

  0%|          | 0/55799 [00:00<?, ?it/s]

CPU times: user 5min 10s, sys: 6min 7s, total: 11min 17s
Wall time: 11min 25s


# 2. Frequent Subgraph Mining (FSM) with gSpan

### Parse Data to Graph

In [22]:
# We define our support using the ratio of 20 for 20k patients from literature
support = int((20/20_000)*55799)
min_subgraph_len = 3
max_subgraph_len = 6

In [25]:
arg_string = f"-fp '{file_paths[0].parent}' -s {support} -l {min_subgraph_len} -u {max_subgraph_len}"
args = parse_args(shlex.split(arg_string))

In [26]:
args

Namespace(gspan_fp=PosixPath('/home/gmatlin/tmp/gspan_output'), support=55, lower=3, upper=6)

In [27]:
%%time
# TODO: SPEED UP REPORTING IN GRAPH MINING
# TODO: ADD PRINT STATEMENTS TO INDICATE HOW FAR ALONG
# TODO: TQDM FOR THE REPORT LOOPS
# TODO: PANDAS ALTERNATIVE FOR THE REPORTING SLOG??

graph_miner = main(args) # WARNING: TAKES A FEW HOURS (?)

Running `gSpan` package on each `.gSpan` file:: 0it [00:00, ?it/s]

Read:	8.41 s
Mine:	51.81999999999999 s
Total:	60.23 s
Read:	8.16 s
Mine:	49.370000000000005 s
Total:	57.53 s
Read:	8.92 s
Mine:	51.629999999999995 s
Total:	60.55 s
Read:	9.94 s
Mine:	46.18 s
Total:	56.12 s
Read:	10.5 s
Mine:	46.78 s
Total:	57.28 s
Read:	3.81 s
Mine:	46.82 s
Total:	50.63 s


Building a report from each column result:   0%|          | 0/6 [00:00<?, ?it/s]

CONCAT DATA
CPU times: user 34min 58s, sys: 6.85 s, total: 35min 5s
Wall time: 35min 6s


##### Save `graph_miner`

In [28]:
with open(f"/home/gmatlin/tmp/{site_name}_{run_id}_graph_miner.pkl", "wb") as dill_file:
    pickle.dump(graph_miner, dill_file)

In [29]:
##### Load `graph_miner`